In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 50)
from scipy.spatial import cKDTree
from math import *
np.random.seed(123)
import urllib.request
import urllib, os

In [2]:
import pandas as pd
import xarray as xr
import math
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta

In [1]:
import numpy as np
import xarray as xr
import xesmf as xe

In [4]:
df_prep = xr.open_dataset(r'/home/shubham/ugp/cleaned/zos_AVISO.nc')

In [5]:
df_prep

<xarray.Dataset>
Dimensions:    (time: 219, bnds: 2, lat: 180, lon: 360)
Coordinates:
  * time       (time) datetime64[ns] 1992-10-16T12:00:00 ... 2010-12-16T12:00:00
  * lat        (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * lon        (lon) float64 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] ...
    lat_bnds   (lat, bnds) float64 ...
    lon_bnds   (lon, bnds) float64 ...
    zos        (time, lat, lon) float32 ...
Attributes: (12/28)
    institution:         Centre National d'Etudes Spatiales
    institute_id:        CNES
    experiment_id:       obs
    source:              SSALTO/DUACS
    model_id:            Obs-AVISO
    contact:             AVISO User Service, aviso@oceanobs.com
    ...                  ...
    Conventions:         CF-1.4
    project_id:          obs4MIPs
    table_id:            Table Omon_obs (31 January 2011) 3852a2b3aff8bddd40f...
    title:               Obs-AVISO model output prepared for obs4MIPs NASA-JP...
    modeling_realm:      ocean
    cmor_version:        2.5.3

In [6]:
df_prep["slice"] = df_prep["zos"].isel(time=0)
df_prep

<xarray.Dataset>
Dimensions:    (time: 219, bnds: 2, lat: 180, lon: 360)
Coordinates:
  * time       (time) datetime64[ns] 1992-10-16T12:00:00 ... 2010-12-16T12:00:00
  * lat        (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * lon        (lon) float64 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] 1992-10-01 1992-11-01 ... 2011-01-01
    lat_bnds   (lat, bnds) float64 -90.0 -89.0 -89.0 -88.0 ... 89.0 89.0 90.0
    lon_bnds   (lon, bnds) float64 0.0 1.0 1.0 2.0 ... 358.0 359.0 359.0 360.0
    zos        (time, lat, lon) float32 ...
    slice      (lat, lon) float32 ...
Attributes: (12/28)
    institution:         Centre National d'Etudes Spatiales
    institute_id:        CNES
    experiment_id:       obs
    source:              SSALTO/DUACS
    model_id:            Obs-AVISO
    contact:             AVISO User Service, aviso@oceanobs.com
    ...                  ...
    Conventions:         CF-1.4
    project_id:          obs4MIPs
    table_id:            Table Omon_obs (31 January 2011) 3852a2b3aff8bddd40f...
    title:               Obs-AVISO model output prepared for obs4MIPs NASA-JP...
    modeling_realm:      ocean
    cmor_version:        2.5.3

In [7]:
df_prep_out = xr.Dataset(
    {
        "lat": (["lat"], np.arange(-90,90, 2.0)),
        "lon": (["lon"], np.arange(0, 360, 2)),
    }
)

regridder = xe.Regridder(df_prep, df_prep_out, 'bilinear')
regridder

Overwrite existing file: bilinear_180x360_90x180.nc 
 You can set reuse_weights=True to save computing time.


xESMF Regridder 
Regridding algorithm:       bilinear 
Weight filename:            bilinear_180x360_90x180.nc 
Reuse pre-computed weights? False 
Input grid shape:           (180, 360) 
Output grid shape:          (90, 180) 
Output grid dimension name: ('lat', 'lon') 
Periodic in longitude?      False

In [8]:
df_prep = df_prep.drop(['time_bnds','lat_bnds','lon_bnds'])

In [9]:
df_out = regridder(df_prep)
df_out

using dimensions ('lat', 'lon') from data variable zos as the horizontal dimensions for this dataset.


/home/shubham/anaconda3/envs/ugp_env/lib/python3.7/site-packages/xesmf/frontend.py:486: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  keep_attrs=keep_attrs


<xarray.Dataset>
Dimensions:  (time: 219, lat: 90, lon: 180)
Coordinates:
  * time     (time) datetime64[ns] 1992-10-16T12:00:00 ... 2010-12-16T12:00:00
  * lon      (lon) int64 0 2 4 6 8 10 12 14 ... 344 346 348 350 352 354 356 358
  * lat      (lat) float64 -90.0 -88.0 -86.0 -84.0 -82.0 ... 82.0 84.0 86.0 88.0
Data variables:
    zos      (time, lat, lon) float64 0.0 0.0 0.0 0.0 0.0 ... nan nan nan nan
    slice    (lat, lon) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... nan nan nan nan nan
Attributes:
    regrid_method:  bilinear

In [10]:
df_out = df_out.drop(['slice'])

In [11]:
df_out['time'][114:]

<xarray.DataArray 'time' (time: 105)>
array(['2002-04-16T00:00:00.000000000', '2002-05-16T12:00:00.000000000',
       '2002-06-16T00:00:00.000000000', '2002-07-16T12:00:00.000000000',
       '2002-08-16T12:00:00.000000000', '2002-09-16T00:00:00.000000000',
       '2002-10-16T12:00:00.000000000', '2002-11-16T00:00:00.000000000',
       '2002-12-16T12:00:00.000000000', '2003-01-16T12:00:00.000000000',
       '2003-02-15T00:00:00.000000000', '2003-03-16T12:00:00.000000000',
       '2003-04-16T00:00:00.000000000', '2003-05-16T12:00:00.000000000',
       '2003-06-16T00:00:00.000000000', '2003-07-16T12:00:00.000000000',
       '2003-08-16T12:00:00.000000000', '2003-09-16T00:00:00.000000000',
       '2003-10-16T12:00:00.000000000', '2003-11-16T00:00:00.000000000',
       '2003-12-16T12:00:00.000000000', '2004-01-16T12:00:00.000000000',
       '2004-02-15T12:00:00.000000000', '2004-03-16T12:00:00.000000000',
       '2004-04-16T00:00:00.000000000', '2004-05-16T12:00:00.000000000',
       '2004-06-16T00:00:00.000000000', '2004-07-16T12:00:00.000000000',
       '2004-08-16T12:00:00.000000000', '2004-09-16T00:00:00.000000000',
       '2004-10-16T12:00:00.000000000', '2004-11-16T00:00:00.000000000',
       '2004-12-16T12:00:00.000000000', '2005-01-16T12:00:00.000000000',
       '2005-02-15T00:00:00.000000000', '2005-03-16T12:00:00.000000000',
       '2005-04-16T00:00:00.000000000', '2005-05-16T12:00:00.000000000',
       '2005-06-16T00:00:00.000000000', '2005-07-16T12:00:00.000000000',
       '2005-08-16T12:00:00.000000000', '2005-09-16T00:00:00.000000000',
       '2005-10-16T12:00:00.000000000', '2005-11-16T00:00:00.000000000',
       '2005-12-16T12:00:00.000000000', '2006-01-16T12:00:00.000000000',
       '2006-02-15T00:00:00.000000000', '2006-03-16T12:00:00.000000000',
       '2006-04-16T00:00:00.000000000', '2006-05-16T12:00:00.000000000',
       '2006-06-16T00:00:00.000000000', '2006-07-16T12:00:00.000000000',
       '2006-08-16T12:00:00.000000000', '2006-09-16T00:00:00.000000000',
       '2006-10-16T12:00:00.000000000', '2006-11-16T00:00:00.000000000',
       '2006-12-16T12:00:00.000000000', '2007-01-16T12:00:00.000000000',
       '2007-02-15T00:00:00.000000000', '2007-03-16T12:00:00.000000000',
       '2007-04-16T00:00:00.000000000', '2007-05-16T12:00:00.000000000',
       '2007-06-16T00:00:00.000000000', '2007-07-16T12:00:00.000000000',
       '2007-08-16T12:00:00.000000000', '2007-09-16T00:00:00.000000000',
       '2007-10-16T12:00:00.000000000', '2007-11-16T00:00:00.000000000',
       '2007-12-16T12:00:00.000000000', '2008-01-16T12:00:00.000000000',
       '2008-02-15T12:00:00.000000000', '2008-03-16T12:00:00.000000000',
       '2008-04-16T00:00:00.000000000', '2008-05-16T12:00:00.000000000',
       '2008-06-16T00:00:00.000000000', '2008-07-16T12:00:00.000000000',
       '2008-08-16T12:00:00.000000000', '2008-09-16T00:00:00.000000000',
       '2008-10-16T12:00:00.000000000', '2008-11-16T00:00:00.000000000',
       '2008-12-16T12:00:00.000000000', '2009-01-16T12:00:00.000000000',
       '2009-02-15T00:00:00.000000000', '2009-03-16T12:00:00.000000000',
       '2009-04-16T00:00:00.000000000', '2009-05-16T12:00:00.000000000',
       '2009-06-16T00:00:00.000000000', '2009-07-16T12:00:00.000000000',
       '2009-08-16T12:00:00.000000000', '2009-09-16T00:00:00.000000000',
       '2009-10-16T12:00:00.000000000', '2009-11-16T00:00:00.000000000',
       '2009-12-16T12:00:00.000000000', '2010-01-16T12:00:00.000000000',
       '2010-02-15T00:00:00.000000000', '2010-03-16T12:00:00.000000000',
       '2010-04-16T00:00:00.000000000', '2010-05-16T12:00:00.000000000',
       '2010-06-16T00:00:00.000000000', '2010-07-16T12:00:00.000000000',
       '2010-08-16T12:00:00.000000000', '2010-09-16T00:00:00.000000000',
       '2010-10-16T12:00:00.000000000', '2010-11-16T00:00:00.000000000',
       '2010-12-16T12:00:00.000000000'], dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2002-04-16 ... 2010-12-16T12:00:00
Attributes:
    bo

In [12]:
df_out = df_out['zos'][114:][:][:]

In [13]:
df_out

<xarray.DataArray 'zos' (time: 105, lat: 90, lon: 180)>
array([[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan],
        ...,
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan],
        ...,
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan],
        ...,
...
        ...,
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan],
        ...,
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan],
        ...,
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan]]])
Coordinates:
  * time     (time) datetime64[ns] 2002-04-16 ... 2010-12-16T12:00:00
  * lon      (lon) int64 0 2 4 6 8 10 12 14 ... 344 346 348 350 352 354 356 358
  * lat      (lat) float64 -90.0 -88.0 -86.0 -84.0 -82.0 ... 82.0 84.0 86.0 88.0

In [14]:
pd.to_datetime(df_out['time'])[0]

Timestamp('2002-04-16 00:00:00')

In [15]:
new_index = pd.period_range(pd.to_datetime(df_out['time'])[0],pd.to_datetime(df_out['time'])[-1],freq='M')

In [16]:
new_index = pd.DataFrame(new_index)
new_index

,0
0,2002-04
1,2002-05
2,2002-06
3,2002-07
4,2002-08
...,...
100,2010-08
101,2010-09
102,2010-10
103,2010-11


In [17]:
df_out

<xarray.DataArray 'zos' (time: 105, lat: 90, lon: 180)>
array([[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan],
        ...,
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan],
        ...,
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan],
        ...,
...
        ...,
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan],
        ...,
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan],
        ...,
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan]]])
Coordinates:
  * time     (time) datetime64[ns] 2002-04-16 ... 2010-12-16T12:00:00
  * lon      (lon) int64 0 2 4 6 8 10 12 14 ... 344 346 348 350 352 354 356 358
  * lat      (lat) float64 -90.0 -88.0 -86.0 -84.0 -82.0 ... 82.0 84.0 86.0 88.0

In [18]:
type(new_index[0][0])

pandas._libs.tslibs.period.Period

In [19]:
new_index = new_index[0].dt.to_timestamp()
type(new_index)

pandas.core.series.Series

In [20]:
new_index

0     2002-04-01
1     2002-05-01
2     2002-06-01
3     2002-07-01
4     2002-08-01
         ...    
100   2010-08-01
101   2010-09-01
102   2010-10-01
103   2010-11-01
104   2010-12-01
Name: 0, Length: 105, dtype: datetime64[ns]

In [21]:
df_out['time'] = pd.to_datetime(df_out['time']).strftime("%Y-%m")

In [22]:
df_out['time'] = pd.to_datetime(df_out['time'])
df_out

<xarray.DataArray 'zos' (time: 105, lat: 90, lon: 180)>
array([[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan],
        ...,
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan],
        ...,
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan],
        ...,
...
        ...,
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan],
        ...,
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan],
        ...,
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan]]])
Coordinates:
  * time     (time) datetime64[ns] 2002-04-01 2002-05-01 ... 2010-12-01
  * lon      (lon) int64 0 2 4 6 8 10 12 14 ... 344 346 348 350 352 354 356 358
  * lat      (lat) float64 -90.0 -88.0 -86.0 -84.0 -82.0 ... 82.0 84.0 86.0 88.0

In [23]:
df_out['time'][:]

<xarray.DataArray 'time' (time: 105)>
array(['2002-04-01T00:00:00.000000000', '2002-05-01T00:00:00.000000000',
       '2002-06-01T00:00:00.000000000', '2002-07-01T00:00:00.000000000',
       '2002-08-01T00:00:00.000000000', '2002-09-01T00:00:00.000000000',
       '2002-10-01T00:00:00.000000000', '2002-11-01T00:00:00.000000000',
       '2002-12-01T00:00:00.000000000', '2003-01-01T00:00:00.000000000',
       '2003-02-01T00:00:00.000000000', '2003-03-01T00:00:00.000000000',
       '2003-04-01T00:00:00.000000000', '2003-05-01T00:00:00.000000000',
       '2003-06-01T00:00:00.000000000', '2003-07-01T00:00:00.000000000',
       '2003-08-01T00:00:00.000000000', '2003-09-01T00:00:00.000000000',
       '2003-10-01T00:00:00.000000000', '2003-11-01T00:00:00.000000000',
       '2003-12-01T00:00:00.000000000', '2004-01-01T00:00:00.000000000',
       '2004-02-01T00:00:00.000000000', '2004-03-01T00:00:00.000000000',
       '2004-04-01T00:00:00.000000000', '2004-05-01T00:00:00.000000000',
       '2004-06-01T00:00:00.000000000', '2004-07-01T00:00:00.000000000',
       '2004-08-01T00:00:00.000000000', '2004-09-01T00:00:00.000000000',
       '2004-10-01T00:00:00.000000000', '2004-11-01T00:00:00.000000000',
       '2004-12-01T00:00:00.000000000', '2005-01-01T00:00:00.000000000',
       '2005-02-01T00:00:00.000000000', '2005-03-01T00:00:00.000000000',
       '2005-04-01T00:00:00.000000000', '2005-05-01T00:00:00.000000000',
       '2005-06-01T00:00:00.000000000', '2005-07-01T00:00:00.000000000',
       '2005-08-01T00:00:00.000000000', '2005-09-01T00:00:00.000000000',
       '2005-10-01T00:00:00.000000000', '2005-11-01T00:00:00.000000000',
       '2005-12-01T00:00:00.000000000', '2006-01-01T00:00:00.000000000',
       '2006-02-01T00:00:00.000000000', '2006-03-01T00:00:00.000000000',
       '2006-04-01T00:00:00.000000000', '2006-05-01T00:00:00.000000000',
       '2006-06-01T00:00:00.000000000', '2006-07-01T00:00:00.000000000',
       '2006-08-01T00:00:00.000000000', '2006-09-01T00:00:00.000000000',
       '2006-10-01T00:00:00.000000000', '2006-11-01T00:00:00.000000000',
       '2006-12-01T00:00:00.000000000', '2007-01-01T00:00:00.000000000',
       '2007-02-01T00:00:00.000000000', '2007-03-01T00:00:00.000000000',
       '2007-04-01T00:00:00.000000000', '2007-05-01T00:00:00.000000000',
       '2007-06-01T00:00:00.000000000', '2007-07-01T00:00:00.000000000',
       '2007-08-01T00:00:00.000000000', '2007-09-01T00:00:00.000000000',
       '2007-10-01T00:00:00.000000000', '2007-11-01T00:00:00.000000000',
       '2007-12-01T00:00:00.000000000', '2008-01-01T00:00:00.000000000',
       '2008-02-01T00:00:00.000000000', '2008-03-01T00:00:00.000000000',
       '2008-04-01T00:00:00.000000000', '2008-05-01T00:00:00.000000000',
       '2008-06-01T00:00:00.000000000', '2008-07-01T00:00:00.000000000',
       '2008-08-01T00:00:00.000000000', '2008-09-01T00:00:00.000000000',
       '2008-10-01T00:00:00.000000000', '2008-11-01T00:00:00.000000000',
       '2008-12-01T00:00:00.000000000', '2009-01-01T00:00:00.000000000',
       '2009-02-01T00:00:00.000000000', '2009-03-01T00:00:00.000000000',
       '2009-04-01T00:00:00.000000000', '2009-05-01T00:00:00.000000000',
       '2009-06-01T00:00:00.000000000', '2009-07-01T00:00:00.000000000',
       '2009-08-01T00:00:00.000000000', '2009-09-01T00:00:00.000000000',
       '2009-10-01T00:00:00.000000000', '2009-11-01T00:00:00.000000000',
       '2009-12-01T00:00:00.000000000', '2010-01-01T00:00:00.000000000',
       '2010-02-01T00:00:00.000000000', '2010-03-01T00:00:00.000000000',
       '2010-04-01T00:00:00.000000000', '2010-05-01T00:00:00.000000000',
       '2010-06-01T00:00:00.000000000', '2010-07-01T00:00:00.000000000',
       '2010-08-01T00:00:00.000000000', '2010-09-01T00:00:00.000000000',
       '2010-10-01T00:00:00.000000000', '2010-11-01T00:00:00.000000000',
       '2010-12-01T00:00:00.000000000'], dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2002-04-01 2002-05-01 ... 2010-12-01

In [24]:
new_index

0     2002-04-01
1     2002-05-01
2     2002-06-01
3     2002-07-01
4     2002-08-01
         ...    
100   2010-08-01
101   2010-09-01
102   2010-10-01
103   2010-11-01
104   2010-12-01
Name: 0, Length: 105, dtype: datetime64[ns]

In [25]:
df_out['lat']

<xarray.DataArray 'lat' (lat: 90)>
array([-90., -88., -86., -84., -82., -80., -78., -76., -74., -72., -70., -68.,
       -66., -64., -62., -60., -58., -56., -54., -52., -50., -48., -46., -44.,
       -42., -40., -38., -36., -34., -32., -30., -28., -26., -24., -22., -20.,
       -18., -16., -14., -12., -10.,  -8.,  -6.,  -4.,  -2.,   0.,   2.,   4.,
         6.,   8.,  10.,  12.,  14.,  16.,  18.,  20.,  22.,  24.,  26.,  28.,
        30.,  32.,  34.,  36.,  38.,  40.,  42.,  44.,  46.,  48.,  50.,  52.,
        54.,  56.,  58.,  60.,  62.,  64.,  66.,  68.,  70.,  72.,  74.,  76.,
        78.,  80.,  82.,  84.,  86.,  88.])
Coordinates:
  * lat      (lat) float64 -90.0 -88.0 -86.0 -84.0 -82.0 ... 82.0 84.0 86.0 88.0

In [26]:
df_out

<xarray.DataArray 'zos' (time: 105, lat: 90, lon: 180)>
array([[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan],
        ...,
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan],
        ...,
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan],
        ...,
...
        ...,
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan],
        ...,
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan],
        ...,
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan]]])
Coordinates:
  * time     (time) datetime64[ns] 2002-04-01 2002-05-01 ... 2010-12-01
  * lon      (lon) int64 0 2 4 6 8 10 12 14 ... 344 346 348 350 352 354 356 358
  * lat      (lat) float64 -90.0 -88.0 -86.0 -84.0 -82.0 ... 82.0 84.0 86.0 88.0

In [27]:
x = df_out.reindex({"time": new_index})
x

<xarray.DataArray 'zos' (time: 105, lat: 90, lon: 180)>
array([[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan],
        ...,
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan],
        ...,
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan],
        ...,
...
        ...,
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan],
        ...,
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan],
        ...,
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan],
        [ 0., nan, nan, ..., nan, nan, nan]]])
Coordinates:
  * time     (time) datetime64[ns] 2002-04-01 2002-05-01 ... 2010-12-01
  * lon      (lon) int64 0 2 4 6 8 10 12 14 ... 344 346 348 350 352 354 356 358
  * lat      (lat) float64 -90.0 -88.0 -86.0 -84.0 -82.0 ... 82.0 84.0 86.0 88.0

In [28]:
x.to_netcdf('/home/shubham/ugp/cl_new/regrid_2_deg/zos_AVISO_04-2002_12-2010.nc')